<a href="https://colab.research.google.com/github/shashwat-j/T5-Translation-Model/blob/main/Hinglish_T5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving hinglish_dataset_json.json to hinglish_dataset_json.json


In [3]:
!pip install transformers datasets

from transformers import AutoConfig, AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, HfArgumentParser
from datasets import load_dataset
from transformers import default_data_collator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [4]:
# Model Setup
model_name = "t5-small"
config = AutoConfig.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, config=config)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
import random

In [33]:
# Load Dataset
raw_dataset = load_dataset("json", data_files={"train": "hinglish_dataset_json.json"})

sample_size = 20000  # Number of random samples to select
total_rows = len(raw_dataset['train'])  # Total number of rows in the dataset
random_indices = random.sample(range(total_rows), sample_size)
data_subset = raw_dataset['train'].select(random_indices)

# Reserve 10% of the data for testing
train_test_split = data_subset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [34]:
# Preprocessing for Hinglish to English Translation
source_prefix = "Translate Hinglish to English: "
source_lang = "hi_ng"
target_lang = "en"
max_source_length = 128
max_target_length = 128
padding = "max_length"

def preprocess(source_data):
    inputs = [source_prefix + sample[source_lang] for sample in source_data["translation"]]
    targets = [sample[target_lang] for sample in source_data["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)
    labels = tokenizer(text_target=targets, max_length=max_target_length, padding=padding, truncation=True)
    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
    ]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply Preprocessing
train_dataset = train_dataset.map(preprocess, batched=True, remove_columns="translation")
test_dataset = test_dataset.map(preprocess, batched=True, remove_columns="translation")

# Data Collator
data_collator = default_data_collator

Map:   0%|          | 0/18000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [35]:
# Training Arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/My Drive/my-t5-hinglish-to-english",  # Directory to save the model
    overwrite_output_dir=True,
    do_train=True,
    per_device_train_batch_size=8,
    num_train_epochs=3,  # will increase later
    save_steps=1000,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
    evaluation_strategy="no"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [36]:
# Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator
)

<ipython-input-36-45dca304c923>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [37]:
# Train the Model
trainer.train()
trainer.save_model()

Step,Training Loss
100,2.634200
200,2.302500
300,2.121400
400,2.006600
500,1.946900
600,1.939800
700,1.882600
800,1.829500
900,1.812800
1000,1.830000


In [40]:
# Inference Example
model = AutoModelForSeq2SeqLM.from_pretrained("my-t5-hinglish-to-english")
input_text = "Translate Hinglish to English: aj weather kaisa hai?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids
outputs = model.generate(input_ids, max_new_tokens=50)
print("Test Output: ", tokenizer.decode(outputs[0], skip_special_tokens=True))


Test Output:  What is the weather?


In [22]:
# Reserve 20% of the data for testing
# train_test_split = raw_dataset['train'].select(range(sample_size)).train_test_split(test_size=0.2)
# train_dataset = train_test_split['train']
# test_dataset = train_test_split['test']

# Preprocess test data
# test_dataset = test_dataset.map(preprocess, batched=True, remove_columns="translation")


Map:   0%|          | 0/600 [00:00<?, ? examples/s]

In [24]:
!pip install evaluate
!pip install torch
import torch
from evaluate import load

In [26]:
import numpy as np

In [41]:
# Load the BLEU metric
bleu = load("bleu")


# Function to generate predictions
def generate_predictions(model, tokenizer, test_dataset):
    predictions = []
    references = []
    for i, example in enumerate(test_dataset):
        try:
            # Instead of tokenizing input_ids again, use them directly
            input_ids = torch.tensor([example["input_ids"]])  # Convert to tensor
            generated_ids = model.generate(input_ids, max_new_tokens=50)
            prediction = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

            # Decode labels for reference
            # Filter out -100 from labels before decoding using numpy
            labels = np.array(example["labels"])
            labels = labels[labels != -100]
            if i%100==0:
              print(f"Processing example {i}: Labels before decode: {labels}")  # Debug print
            reference = tokenizer.decode(labels.tolist(), skip_special_tokens=True)
            if i%100==0:
              print(f"Processing example {i}: Reference decoded: {reference}")  # Debug print

            predictions.append(prediction)
            references.append(reference)
        except OverflowError as e:
            print(f"OverflowError occurred at example {i}: {e}")
            print(f"Problematic labels: {example['labels']}")
            break  # Stop processing if OverflowError occurs

    return predictions, references

# Generate predictions and references
predictions, references = generate_predictions(model, tokenizer, test_dataset)

# Calculate BLEU score
results = bleu.compute(predictions=predictions, references=references)
print(results)


Processing example 0: Labels before decode: [ 577   72   57   48 2377    1]
Processing example 0: Reference decoded: play more by this artist
Processing example 100: Labels before decode: [1072   25  356   82   97   52   21  140   12  129  335  996  676   13
 2085    1]
Processing example 100: Reference decoded: Can you set my timer for me to get 10 extra minutes of sleep
Processing example 200: Labels before decode: [ 366   19    8 2419    7 3397   16 8912 3552   48  215    1]
Processing example 200: Reference decoded: When is the Blues Festival in Clearwater this year
Processing example 300: Labels before decode: [5607  140   12  830    8 2959 7481    1]
Processing example 300: Reference decoded: remind me to bring the foreign currency
Processing example 400: Labels before decode: [    3  6058    27  9132 18111  8043   489  5422 16309    27   283 17253
  2344  3001   446  9195   567   377   480  5332  4171 19409     3 18375
 14536   272   476   335  5422   350  8087   567  1853   205